In [1]:
#%matplotlib notebook
#%matplotlib inline
# Importing necessary packages:
from glob import glob
import pandas as pd
import seaborn as sns
from PipeLine import *

In [ ]:
def log_outputs(logs_path, path_to_save='./'):
    outname = logs_path.split("/")[1]
    dirpath = os.path.join(outdir,"log_summary")
    try:
        os.mkdir(dirpath)
    except (FileExistsError, OSError):
        print ("The directory %s exits or is failed to create." % dirpath)
    else:
        print ("Successfully created the directory %s " % dirpath)
    # if dangerous build needed 
    details_out = dirpath + "/" + outname + "_details.csv"
    with open(details_out, "w") as detailsfile:
        # neigh_modify delay NUM every NUM check YES/NO:
        detailsfile.write('groupname,filename,ens,run_seg,rskin,delay,every,check,epsilon,dcrowd,ncrowd,lcyl,dcyl,nmon,total_time_s,cores,timestep,atoms,ts_per_sec,')
        # Section columns: min time, avg time, max time, %varavg, %total"
        # Section rows: Pair, Bond, Neigh, Comm, Output, Modify, Other
        detailsfile.write('pair_avg_s,pair_pct,bond_avg_s,bond_pct,neigh_avg_s,neigh_pct,comm_avg_s,comm_pct,output_avg_s,output_pct,modify_avg_s,modify_pct,other_avg_s,other_pct,dangerous\n')
    
    
    runtime_out = dirpath + "/" + outname + "_runtime.csv"
    with open(runtime_out, "w") as runfile:
        runfile.write('groupname,filename,ncores,natoms,wall_time\n')
        runfile.close()
    return details_out , runtime_out

def lammps_log_details(log_files, details_out , runtime_out):
    for file in log_files:
        filename = file[0].split('.log')
        filename = filename[0]
        filename = filename.split('/')[-1]
        ens = filename.split('ens')[-1]
        groupname = filename.split('ens')[0]
        with open(file[0],'r') as log,\
        open(details_out, "a") as detailsfile,\
        open(runtime_out, "a") as runfile:   
            line = log.readline()
            # The other of while loop are important
            #neigh_modify delay NUM every NUM check YES/NO page NUM one NUM: 
            j = 1
            while line:

                while line.startswith('variable'):
                    words = line.split()
                    line = log.readline()
                    if words[1] == 'epsilon1':
                        epsilon = words[3]

                    if words[1] == 'sig2':
                        dcrowd = words[3]  

                    if words[1] == 'n_crowd':
                        ncrowd = words[3] 

                    if words[1] == 'lz':
                        lcyl = str(2*float(words[3]))  

                    if words[1] == 'r':
                        dcyl = str(2*float(words[3]))

                    if words[1] == 'n_bug':
                        nmon = words[3]


                if line.startswith('neighbor'):
                    words = line.split()
                    rskin = words[1].strip() # rskin

                #neigh_modify delay NUM every NUM check YES/NO page NUM one NUM:  
                if line.startswith('neigh_modify'):
                    words = line.split()
                    # picking the NUMs and Yes/No from neigh_modify command
                    delay = words[2].strip()
                    every = words[4].strip()
                    check = words[6].strip()


                if line.startswith('Loop time'):
                    detailsfile.write(groupname)
                    detailsfile.write(",")
                    detailsfile.write(filename)
                    detailsfile.write(",")
                    detailsfile.write(ens)
                    detailsfile.write(",")
                    detailsfile.write(str(j))#total time
                    detailsfile.write(",")
                    j += 1

                    # neighbor and neigh_modify ocurres  one time but other occures 15 times.
                    detailsfile.write(rskin) # rskin
                    detailsfile.write(",")
                    detailsfile.write(delay) # delay
                    detailsfile.write(",")
                    detailsfile.write(every) # every
                    detailsfile.write(",")
                    detailsfile.write(check) # check
                    detailsfile.write(",")

                    detailsfile.write(epsilon) # epsilon
                    detailsfile.write(",")
                    detailsfile.write(dcrowd) # dcrowd
                    detailsfile.write(",")
                    detailsfile.write(ncrowd) # ncrowd
                    detailsfile.write(",")
                    detailsfile.write(lcyl) # lcyl
                    detailsfile.write(",")
                    detailsfile.write(dcyl) # dcyl
                    detailsfile.write(",")
                    detailsfile.write(nmon) # nmon
                    detailsfile.write(",")


                    words = line.split()
                    detailsfile.write(words[3].strip())#total time
                    detailsfile.write(",")
                    ncores = words[5].strip()
                    detailsfile.write(ncores)# # of cores
                    detailsfile.write(",")
                    detailsfile.write(words[8].strip())# total timesteps
                    detailsfile.write(",")
                    natoms = words[11].strip()
                    detailsfile.write(natoms)# total atoms
                    detailsfile.write(",")                          

                if line.startswith('Performance:'):
                    words = line.split()
                    detailsfile.write(words[3].strip())# timesteps per second
                    detailsfile.write(",")

                if line.startswith('Section'):
                    _ = log.readline()
                    for i in range(6): # Section rows: Pair, Bond, Neigh, Comm, Output, Modify, Other
                        # Section columns: min time, avg time, max time, %varavg, %total"
                        line = log.readline()
                        sect_min = line.split('|')[2].strip()
                        detailsfile.write(sect_min)
                        detailsfile.write(",")

                        sect_pct = line.split()[-1] # Pair pct of total time
                        detailsfile.write(sect_pct)
                        detailsfile.write(",")

                    line = log.readline()
                    sect_min = line.split('|')[2].strip()
                    detailsfile.write(sect_min)
                    detailsfile.write(",")
                    sect_pct = line.split()[-1] # Pair pct of total time
                    detailsfile.write(sect_pct)
                    detailsfile.write(",")

                if line.startswith('Dangerous'):
                    words = line.split()
                    detailsfile.write(str(int(words[-1]))) # # number of dangerous builds
                    #detailsfile.write(",")
                    detailsfile.write("\n")

                # runtime files
                if line.startswith('Total wall time'):
                    runfile.write(groupname)
                    runfile.write(",")
                    runfile.write(filename)
                    runfile.write(",")
                    runfile.write(ncores)
                    runfile.write(",")
                    runfile.write(natoms)
                    runfile.write(",")
                    words = line.split()
                    runfile.write(words[-1]) # total wall time
                    runfile.write("\n")

                line = log.readline()



In [ ]:
def log_outputs(log_file, geometry):
    cell_attrs = PipeLine.cellAttributes(log_file,geometry=geometry)
    output = 'N'+str(cell_attrs.nmon)+'D'+str(cell_attrs.dcyl)+'ac'+str(cell_attrs.dcrowd)
    # if dangerous build needed 
    details_out = output + "-log_details.csv"
    with open(details_out, "w") as detailsfile:
        # neigh_modify delay NUM every NUM check YES/NO:
        detailsfile.write('groupname,filename,ens,run_seg,rskin,delay,every,check,epsilon,dcrowd,ncrowd,lcyl,dcyl,nmon,total_time_s,cores,timestep,atoms,ts_per_sec,')
        # Section columns: min time, avg time, max time, %varavg, %total"
        # Section rows: Pair, Bond, Neigh, Comm, Output, Modify, Other
        detailsfile.write('pair_avg_s,pair_pct,bond_avg_s,bond_pct,neigh_avg_s,neigh_pct,comm_avg_s,comm_pct,output_avg_s,output_pct,modify_avg_s,modify_pct,other_avg_s,other_pct,dangerous\n')
    
    
    runtime_out = output + "-log_runtime.csv"
    with open(runtime_out, "w") as runfile:
        runfile.write('groupname,filename,ncores,natoms,wall_time\n')
        runfile.close()
      
    return details_out , runtime_out

In [2]:
log_files = glob('../N*ac2-logs/*.log')
geometry = 'cylinder'
log_files = PipeLine.file_reader(log_files,extensions=['.log'])

Total number of files is  184
Path to the first pair of the sorted file:  ('../N2000D20ac2-logs/N2000epsilon5.0r10.5lz504sig2.0nc7560dt0.005bdump1000adump5000ens1.log',)


In [3]:
details_out , runtime_out = PipeLine.log_outputs(log_files[0][0], geometry=geometry)

This is an ensemble-averaged file, so ensemble number: 0


In [4]:
PipeLine.lammps_log_details(log_files, details_out , runtime_out)

In [ ]:
logs_dirs = glob('../N*ac2-logs/')
path_to_save  = './'
details_files = []
runtime_files = []

for logs_di in log_files:
    logs_path = logs_dir+'/*.log'
    details_out , runtime_out = PipeLine.log_outputs(logs_path, path_to_save='./')
    details_files.append(details_out)
    runtime_files.append(runtime_out)
    PipeLine.lammps_log_details(logs_path, details_out , runtime_out)
    

all_files = details_files + runtime_files
log_file_pairs = PipeLine.file_reader(all_files,extensions=['_runtime.csv','_details.csv'])


In [ ]:
runtime_dfs = []
for log_pairs in log_file_pairs:
    runtime_df = pd.read_csv(log_pairs[0])
    runtime_df.wall_time = pd.to_timedelta(runtime_df.wall_time)
    runtime_df['total_time_hr'] = runtime_df.wall_time.dt.total_seconds() / 3600
    runtime_dfs.append(runtime_df)

df_of_runtimes = pd.concat(runtime_dfs)
#groupname = [filename.split("ens")[0] for filename in runtime_data.filename]
#groupname =list(dict.fromkeys(groupname))
#cols = ['ncores','natoms','groupname']
#runtime_data.groupby(cols)['total_time_hr'].mean()

In [ ]:
data = df_of_runtimes
facet_grid = sns.lmplot(x="natoms", y="total_time_hr", hue='ncores', data=data,legend_out=False)#,height=4, aspect=16/9);
facet_grid.tight_layout()
facet_grid.set_axis_labels("Particles","Wall time (hours)")
facet_grid.legend.set_title("CPUs")
facet_grid.set(xlim=(0, 140000), ylim=(0, 45))
#plt.legend(title="CPUs", loc='upper right')
output="../walltime_vs_natoms.pdf"
facet_grid.savefig(output)

g = sns.FacetGrid(data, col="ncores",sharex=False,sharey=False)
g.map(sns.regplot, "natoms", "total_time_hr")
g.set_titles(col_template="{col_name} CPUs")
g.set_axis_labels("Particles" ,"Wall time (hours)")
#g.axes[0][0].set_xlim((1000, 5000))
#g.axes[0][1].set_xlim((5000, 25000))
#g.axes[0][2].set_xlim((25000, 50000))
#g.axes[0][3].set_xlim((50000, 100000))
#g.axes[0][4].set_xlim((100000,150000))
#g.axes[0][0].set_xlim((1000, 5000))
g.tight_layout()
output="../walltime_vs_natoms_separate.pdf"
g.savefig(output)